## 介绍 

本课程将涵盖: 
- 什么是函数调用及其用途 
- 如何使用Azure OpenAI创建函数调用 
- 如何将函数调用集成到应用程序中 

## 学习目标 

完成本课程后，您将了解并掌握: 

- 使用函数调用的目的 
- 使用Azure Open AI服务设置函数调用 
- 为应用程序的使用情况设计有效的函数调用

## 理解函数调用

在这堂课中，我们希望为我们的教育创业公司开发一个功能，让用户能够使用聊天机器人找到技术课程。我们将推荐符合他们的技能水平、当前角色和感兴趣的技术的课程。

为了完成这个功能，我们将使用以下组合：
- `Azure Open AI` 来为用户创建聊天体验
- `Microsoft Learn Catalog API` 来帮助用户根据他们的请求找到课程
- `函数调用` 来接受用户的查询并将其发送到一个函数以发出API请求。

首先，让我们看看为什么我们首先要使用函数调用：

```python
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response

print(second_response.choices[0].message)
```

### 为什么需要函数调用

如果您已经完成了该课程中的其他课程，那么您可能已经了解到使用大型语言模型（LLM）的强大之处。希望您也能看到它们的一些局限性。

函数调用是Azure Open AI服务的一个功能，旨在克服以下限制：
1）一致的响应格式
2）在聊天上下文中能够使用应用程序的其他数据来源

在函数调用之前，LLM的响应是无结构且不一致的。开发人员需要编写复杂的验证代码，以确保能够处理响应的每个变化。

用户无法获得像“斯德哥尔摩目前的天气是多少？”这样的答案。这是因为模型受限于数据训练的时间。

让我们看一下下面的例子，来说明这个问题：

假设我们想创建一个学生数据的数据库，以便为他们推荐合适的课程。下面有两个学生描述，它们在包含的数据中非常相似。

In [1]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finshing his studies."

```markdown
We want to send this to an LLM to parse the data. This can later be used in our application to send this to an API or store it in a database. 

Let's create two identical prompts that we instruct the LLM on what information that we are interested in: 
```

###prompt
We want to send this to an LLM to parse the parts that are important to our product. So we can create two identical prompts to instruct the LLM: 

In [2]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


创建完这两个提示词后，我们将使用 `openai.ChatCompletion` 将它们发送给LLM。我们将提示词存储在 `messages` 变量中，并将角色分配为 `user`。这样做是为了模拟用户编写的消息发送给聊天机器人。

In [4]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'],
  api_key=os.environ['AZURE_OPENAI_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

ModuleNotFoundError: No module named 'idna'

现在我们可以将这两个请求发送到LLM，并检查我们收到的响应。

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

即使提示相同，描述类似，我们也可能会得到不同格式的`Grades`属性。

如果您多次运行上面的单元格，格式可能是`3.7`或`3.7 GPA`。

这是因为LLM接受书面提示的非结构化数据，并返回非结构化数据。我们需要有一个结构化格式，这样我们在存储或使用这些数据时才知道可以期待什么。

通过使用函数调用，我们可以确保收到结构化的数据。在使用函数调用时，LLM实际上不会调用或运行任何函数。相反，我们为LLM创建一个结构，让它遵循其响应。然后我们使用这些结构化的响应来了解我们应用程序中要运行哪个函数。


![功能调用流程图](./images/Function-Flow.png?WT.mc_id=academic-105485-koreyst)

我们可以将从函数返回的内容发送回LLM。然后LLM会用自然语言回复，以回答用户的查询。

### 使用函数调用的用例

**调用外部工具**
聊天机器人在为用户提供答案时非常出色。通过使用函数调用，聊天机器人可以利用用户的消息来完成特定任务。例如，学生可以要求聊天机器人“给我的导师发送邮件，说明我需要更多关于这个科目的帮助”。这可以通过调用函数`send_email(to: string, body: string)`来实现。

**创建API或数据库查询**
用户可以使用自然语言查找信息，并将其转换为格式化的查询或API请求。例如，教师可以要求“完成了最后一次作业的学生是谁”，这可以调用一个名为`get_completed(student_name: string, assignment: int, current_status: string)`的函数。

**创建结构化数据**
用户可以从一段文本或CSV中提取重要信息，并利用LLM进行处理。例如，学生可以将维基百科关于和平协议的文章转换为人工智能闪卡。这可以通过调用一个名为`get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`的函数来实现。

## 2. 创建您的第一个函数调用

创建函数调用的过程包括3个主要步骤：
1. 使用您的函数列表和用户消息调用聊天完成 API
2. 读取模型的响应以执行一个动作，比如执行一个函数或 API 调用
3. 使用来自您的函数的响应再次调用聊天完成 API，以使用该信息创建对用户的响应。

![函数调用流程](./images/LLM-Flow.png?WT.mc_id=academic-105485-koreyst)

### 函数调用元素

#### 用户输入

第一步是创建用户消息。这可以通过获取文本输入的值动态分配，或者您可以在此处分配一个值。如果这是您第一次使用聊天完成 API，我们需要定义消息的`role`和`content`。

`role`可以是`system`（创建规则）、`assistant`（模型）或`user`（最终用户）。对于函数调用，我们将将其分配为`user`并提供一个示例问题。

In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### 创建函数

接下来，我们将定义一个函数以及该函数的参数。我们只会使用一个名为 `search_courses` 的函数，但你也可以创建多个函数。

**重要提示**：函数包含在发送给LLM的系统消息中，并将消耗你可用的令牌数量。

In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

## 定义

`name` - 我们想要调用的函数的名称。

`description` - 这是函数如何工作的描述。在这里，明确和清晰地表达是很重要的。

`parameters` - 您希望模型在其响应中生成的数值和格式的列表。

`type` - 属性将存储在其中的数据类型。

`properties` - 模型将用于其响应的特定值的列表。

`name` - 模型将在其格式化响应中使用的属性的名称。

`type` - 此属性的数据类型。

`description` - 具体属性的描述。

**可选的**

`required` - 函数调用完成所需的属性。

### 进行函数调用
在定义函数后，我们现在需要在调用聊天完成API时将其包含进去。我们通过将`functions`添加到请求中来实现这一点。 在本例中，`functions = functions`。

还有一个选项可以将`function_call`设置为`auto`。这意味着我们将让LLM根据用户消息决定应调用哪个函数，而不是自行指定。

In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

现在让我们看一下响应并查看其格式： 

```
{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}
```

您可以看到函数的名称被调用，并且从用户消息中，LLM能够找到符合函数参数的数据。

## 3.将函数调用集成到应用程序中

在我们测试了LLM的格式化响应之后，现在我们可以将其集成到一个应用程序中。

### 管理流程

要将这个功能集成到我们的应用程序中，让我们采取以下步骤：

首先，让我们调用Open AI服务并将消息存储在名为`response_message`的变量中。

In [ ]:
response_message = response.choices[0].message

现在我们将定义一个函数，该函数将调用Microsoft Learn API以获取课程列表：

In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



作为最佳实践，我们将查看模型是否需要调用函数。然后，我们将创建其中一个可用的函数，并将其与被调用的函数相匹配。接下来，我们将获取函数的参数，并将它们映射到LLM的参数中。

最后，我们将附加函数调用消息和由`search_courses`消息返回的值。这样LLM就拥有了回应用户所需的所有信息，可以使用自然语言进行回复。

In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



现在我们将发送更新后的消息给LLM，这样我们就可以收到一个自然语言的回复，而不是一个API JSON格式的回复。

In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## 代码挑战

干得好！为了继续学习Azure Open AI函数调用，你可以完成以下任务:

- 添加函数的更多参数，这些参数可能有助于学习者找到更多课程。你可以在这里找到可用的API参数:https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
- 创建另一个函数调用，获取学习者的更多信息，比如他们的母语
- 当函数调用和/或API调用没有返回合适的课程时，创建错误处理

In [ ]:
#初始化
import os
from openai import AzureOpenAI
import json


#读取环境变量
temperature = 0.0
api_base = ""
api_key = ""
api_type = "azure"
api_version = "2024-02-01"
model = "gpt-4-turbo"

client = AzureOpenAI(
    azure_endpoint = api_base,
    azure_deployment = model,
    api_key = api_key,
    api_version = api_version,
    )
    
messages= [
    {"role": "user", "content": "Find beachfront hotels in San Diego for less than $300 a month with free breakfast."}
]

functions= [  
    {
        "name": "search_hotels",
        "description": "Retrieves hotels from the search index based on the parameters provided",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The location of the hotel (i.e. Seattle, WA)"
                },
                "max_price": {
                    "type": "number",
                    "description": "The maximum price for the hotel"
                },
                "features": {
                    "type": "string",
                    "description": "A comma separated list of features (i.e. beachfront, free wifi, etc.)"
                }
            },
            "required": ["location"]
        }
    }
]  

response = client.chat.completions.create(
    model=model, # model = "deployment_name"
    messages= messages,
    functions = functions,
    function_call="auto",
)

response = client.chat.completions.create(
    model=model, # model = "deployment_name"
    messages= messages,
    functions = functions,
    function_call="auto",
)

print(response.choices[0].message.model_dump_json(indent=2))

In [ ]:
# Example function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model=model,
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
print(run_conversation())